# Intiating the Langsmith Tracing

In [1]:
### Langsmith tracking
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")  

# Create wrappers or tools

## 1) Wiki_tool

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikiapi=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=wikiapi)
wiki_tool.name

'wikipedia'

## 2) Arxiv_tool

In [3]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxivapi=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxivapi)
arxiv_tool.name

'arxiv'

## 3) Url_tool(Langsmith webpage)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS
embeddingllm=OllamaEmbeddings(model="llama3.2")

# Load data
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

# Treasform data
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
document=splitter.split_documents(docs)

# Embeded data (covert to vectors)
vectordb=FAISS.from_documents(document,embeddingllm)

# Creating a retriever
retrieve=vectordb.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# Creating a tool from the retriever
from langchain.tools.retriever import create_retriever_tool
url_tool=create_retriever_tool(retrieve,"Langsmith-Search","Search for information about Langsmith. For any questions about langsmith, you must use this tool")
url_tool.name

'Langsmith-Search'

In [6]:
# Now Combining these three models
tools=[wiki_tool,url_tool,arxiv_tool]

# Seting up the llm and prompt for Agent

In [7]:
# Importing the llm
from langchain_ollama import ChatOllama
llm=ChatOllama(model="llama3.2")

# seting up the chat prompt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")

# Creating Agent

In [8]:
from langchain.agents import create_tool_calling_agent
initAgent=create_tool_calling_agent(llm,tools,prompt)

# Agent Executer

In [9]:
from langchain.agents import AgentExecutor
agent=AgentExecutor(agent=initAgent,tools=tools,verbose=True)

In [10]:
# To begin Searches
question="what is langsmith"
result=agent.invoke({"input":question})



> Entering new AgentExecutor chain...

Invoking: `Langsmith-Search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Get started by adding tracing to your application.
Create dashboards to view key metrics like RPS, error rates and costs.

Evals​
The quality and development speed of AI applications depends on high-quality evaluation datasets and metrics to test and optimize your applications on. The LangSmith SDK and UI make building and running high-quality evaluations easy.

Get started by creating your first evaluation.
Quickly assess the performance of your application using our off-the-shelf evaluators as a starting point.
Analyze results of evaluations in the LangSmith UI and compare results over time.
Easily collect human feedback on your data to improve your application.

Prompt Engineering​
While traditional software applications are built by writing code, AI applications involve writing prompts to instruct the LLM on what to do. LangSmith provide

In [11]:
print(result['output'])

Langsmith is a framework-agnostic tool for building and optimizing large language model (LLM) applications. It provides a set of tools and features to enable prompt engineering, evaluation, and observability for LLMs.

Key features of Langsmith include:

* Prompt engineering: enables the creation of high-quality prompts for LLMs
* Evaluation: provides a way to assess the performance of LLM applications using off-the-shelf evaluators and human feedback
* Observability: offers LLM-native observability to gain insights into application performance

Langsmith can be used with or without LangChain's open-source frameworks langchain and langgraph, and is suitable for use in Python, JavaScript/TS, and other programming languages.
